<a href="https://colab.research.google.com/github/danielsaniam/ai-recipe-finder/blob/main/Web_scrapping_recipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Scrapping Recipes

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import load_dataset
from bs4 import BeautifulSoup
import requests
from functools import reduce
from urllib.parse import urlparse


In [3]:
dataset = load_dataset("somosnlp/recetas-cocina")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
df = pd.DataFrame(dataset["train"])

## Data revision

In [5]:
df.head()

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25


In [6]:
df.iloc[0]["url"]

'https://www.mycolombianrecipes.com/es/arepas-de-queso/'

In [7]:
# Function to extract domain from a URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Apply the extract_domain function to the 'url' column and create a set of unique domains
unique_domains = set(df['url'].apply(extract_domain))

In [8]:
unique_domains

{'www.mycolombianrecipes.com',
 'www.recetas.com',
 'www.recetasgratis.net',
 'www.recetasnestle.com.co'}

## Get the data from the different sources

In [9]:
# Define a function to scrape information from a URL
def get_value_by_selector_if_exists(selector, soup):
    span = soup.select(selector)
    if len(span) > 0:
      return span[0].get_text()
    else:
      return ''

## Get data from www.mycolombianrecipes.com

In [10]:
def scrape_info_colombian_recipes(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get description:
        p_desc_tags = soup.select('.bialty-container > p')
        description = ''.join(x.decode_contents() for x in p_desc_tags)

        # Get images:
        img_tags = soup.select('.ns-pinterest-image > img')
        img_urls = [img['data-lazy-src'] for img in img_tags]

        # Get the images for the steps:
        step_img_urls = []

        # Get categories:
        categories = [
            soup.select('.breadcrumb > span > span > a')[2].decode_contents()
        ]

        value = get_value_by_selector_if_exists('.wprm-recipe-course', soup)
        if value != '':
          categories.append(value)

        value = get_value_by_selector_if_exists('.wprm-recipe-cuisine', soup)
        if value != '':
          categories.append(value)

        # NO preparation time:
        preparation_time = ''

        # Get servings
        servings = get_value_by_selector_if_exists('.wprm-recipe-servings', soup)

        # Get keywords:
        keywords = get_value_by_selector_if_exists('.wprm-recipe-keyword', soup)

        # NO difficulty:
        difficulty = ''

        # No side dish:
        side_dish = ''

        # Customize this part to extract the information you need from the webpage
        scraped_data = {
            "description": description,
            "img_urls": img_urls,
            "step_img_urls": step_img_urls,
            "categories": categories,
            "preparation_time": preparation_time,
            "servings": servings,
            "keywords": keywords,
            "difficulty": difficulty,
            "side_dish": side_dish
        }
        return scraped_data
    else:
        raise RuntimeError("The resource at the URL does not exist")
        # return { "description": None, "img_urls": None, "categories": None,
        #          "preparation_time": None, "servings": None, "keywords": None }

In [11]:
specific_domain = 'https://www.mycolombianrecipes.com'

df_filtered = df[df['url'].str.startswith(specific_domain)]

df_expanded = df_filtered['url'].apply(lambda x: pd.Series(scrape_info_colombian_recipes(x)))
df_filtered = df_filtered.join(df_expanded)

## Get data from www.recetas.com

In [12]:
def scrape_info_recetas_com(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get Title:
        title = get_value_by_selector_if_exists('h1[itemprop="name"]', soup)

        # NO description:
        description = ''

        # Get images:
        img_tags = soup.select('.recipe_image > img')
        img_urls = [img_tags[0]['src']]

        # Get the images for the steps:
        step_img_urls = []

        # Get categories:
        categories = []

        breadcrumb_links = soup.select('nav.path > a')
        categories = [x.decode_contents() for x in breadcrumb_links[2:]]

        # Get time preparation:
        preparation_time = get_value_by_selector_if_exists('.time time[itemprop="totalTime"]', soup)

        # Get servings
        value = get_value_by_selector_if_exists('.servings > span', soup)
        servings = value.replace('Comen ', '') if value != '' else ''

        # Get keywords:
        keywords = get_value_by_selector_if_exists('.location a', soup)

        # Get difficulty:
        difficulty = get_value_by_selector_if_exists('.difficulty span', soup)

        # No side dish:
        side_dish = ''

        # Customize this part to extract the information you need from the webpage
        scraped_data = {
            "title": title,
            "description": description,
            "img_urls": img_urls,
            "step_img_urls": step_img_urls,
            "categories": categories,
            "preparation_time": preparation_time,
            "servings": servings,
            "keywords": keywords,
            "difficulty": difficulty,
            "side_dish": side_dish
        }
        return scraped_data
    else:
        raise RuntimeError("The resource at the URL does not exist")
        # return { "title": None, "description": None, "img_urls": None, "categories": None,
        #          "preparation_time": None, "servings": None, "keywords": None }

In [13]:
specific_domain = 'https://www.recetas.com'

df_filtered = df[df['url'].str.startswith(specific_domain)]
df_filtered.drop('title', axis=1, inplace=True)

df_expanded = df_filtered['url'].apply(lambda x: pd.Series(scrape_info_recetas_com(x)))
df_filtered = df_filtered.join(df_expanded)

<ipython-input-13-3af3bf8ab960>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop('title', axis=1, inplace=True)


Get data from www.recetasgratis.net

In [14]:
def scrape_info_recetas_net(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get description:
        description = get_value_by_selector_if_exists('.intro > p', soup)

        # Get images:
        img_tags = soup.select('.imagen_wrap img')
        img_urls = [img_tags[0]['src']]

        # Get images for the steps:
        step_img_urls = [x['src'] for x in img_tags[1:]]

        # Get categories:
        categories = []

        breadcrumb_spans = soup.select('.breadcrumb:first-child > li > a > span')
        categories = [x.decode_contents() for x in breadcrumb_spans[1:]]

        div_aditional_features = soup.select('.properties.inline')[0].get_text().replace('Características adicionales:', '')
        categories = categories + [x.strip() for x in div_aditional_features.split(',')]

        # Get time preparation:
        preparation_time = get_value_by_selector_if_exists('.property.duracion', soup)

        # Get servings
        value = get_value_by_selector_if_exists('.property.comensales', soup)
        servings = value.replace(' comensales', '') if value else ''

        # NO keywords:
        keywords = ''

        # Get difficulty:
        value = get_value_by_selector_if_exists('.property.dificultad', soup)
        difficulty = value.replace('Dificultad ', '') if value else ''

        # Get side dish:
        apartados = soup.select('.apartado')
        side_dish = apartados[-2].decode_contents()

        # Customize this part to extract the information you need from the webpage
        scraped_data = {
            "description": description,
            "img_urls": img_urls,
            "step_img_urls": step_img_urls,
            "categories": categories,
            "preparation_time": preparation_time,
            "servings": servings,
            "keywords": keywords,
            "difficulty": difficulty,
            "side_dish": side_dish
        }
        return scraped_data
    else:
        raise RuntimeError("The resource at the URL does not exist")
        # return { "title": None, "description": None, "img_urls": None, "categories": None,
        #          "preparation_time": None, "servings": None, "keywords": None }

In [15]:
specific_domain = 'https://www.recetasgratis.net'

df_filtered = df[df['url'].str.startswith(specific_domain)]
df_filtered = df_filtered.head(10)

df_expanded = df_filtered['url'].apply(lambda x: pd.Series(scrape_info_recetas_net(x)))
df_filtered = df_filtered.join(df_expanded)

## Get data from www.recetasnestle.com.co (Not working by the moment, waiting for feedback).

In [16]:
'''
def scrape_info_nestle(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # NO description:
        description = ''

        # Get images:
        img_tags = soup.select('img[loading="eager"]')
        img_urls = [img_tags[0]['src']]

        # Get images for the steps:
        step_img_urls = [x['src'] for x in img_tags[1:]]

        # Get categories:
        categories = []

        breadcrumb_spans = soup.select('.breadcrumb:first-child > li > a > span')
        categories = [x.decode_contents() for x in breadcrumb_spans[1:]]

        div_aditional_features = soup.select('.properties.inline')[0].get_text().replace('Características adicionales:', '')
        categories = categories + [x.strip() for x in div_aditional_features.split(',')]

        # Get time preparation:
        preparation_time = get_value_by_selector_if_exists('.property.duracion', soup)

        # Get servings
        value = get_value_by_selector_if_exists('.property.comensales', soup)
        servings = value.replace(' comensales', '') if value else ''

        # NO keywords:
        keywords = ''

        # Get difficulty:
        value = get_value_by_selector_if_exists('.property.dificultad', soup)
        difficulty = value.replace('Dificultad ', '') if value else ''

        # Get side dish:
        apartados = soup.select('.apartado')
        side_dish = apartados[-2].decode_contents()

        # Customize this part to extract the information you need from the webpage
        scraped_data = {
            "description": description,
            "img_urls": img_urls,
            "step_img_urls": step_img_urls,
            "categories": categories,
            "preparation_time": preparation_time,
            "servings": servings,
            "keywords": keywords,
            "difficulty": difficulty,
            "side_dish": side_dish
        }
        return scraped_data
    else:
        raise RuntimeError("The resource at the URL does not exist")
        # return { "title": None, "description": None, "img_urls": None, "categories": None,
        #          "preparation_time": None, "servings": None, "keywords": None }
'''

'\ndef scrape_info_nestle(url):\n    response = requests.get(url)\n    if response.status_code == 200:\n        soup = BeautifulSoup(response.text, \'html.parser\')\n\n        # NO description:\n        description = \'\'\n\n        # Get images:\n        img_tags = soup.select(\'img[loading="eager"]\')\n        img_urls = [img_tags[0][\'src\']]\n\n        # Get images for the steps:\n        step_img_urls = [x[\'src\'] for x in img_tags[1:]]\n\n        # Get categories:\n        categories = []\n\n        breadcrumb_spans = soup.select(\'.breadcrumb:first-child > li > a > span\')\n        categories = [x.decode_contents() for x in breadcrumb_spans[1:]]\n\n        div_aditional_features = soup.select(\'.properties.inline\')[0].get_text().replace(\'Características adicionales:\', \'\')\n        categories = categories + [x.strip() for x in div_aditional_features.split(\',\')]\n\n        # Get time preparation:\n        preparation_time = get_value_by_selector_if_exists(\'.property.durac

In [17]:
specific_domain = 'https://www.recetasnestle.com.co'

df_filtered = df[df['url'].str.startswith(specific_domain)]
df_filtered = df_filtered.head(10)

# df_expanded = df_filtered['url'].apply(lambda x: pd.Series(scrape_info_nestle(x)))
# df_filtered = df_filtered.join(df_expanded)
df_filtered

,title,url,ingredients,steps,uuid
446,Ensalada de atún con papa,https://www.recetasnestle.com.co/recetas/ensal...,1 libra de papa sabanera sin piel y cortada...,1. En una olla con suficiente agua coloca...,None
447,Ensalada mexicana con cochinita pibil,https://www.recetasnestle.com.co/recetas/ensal...,1 libra de pierna de cerdo (500 g) 1 s...,"1. Precalienta el horno a 180°C/ 360°F, 1...",None
448,"Ensalada con chips de berenjena al horno, y qu...",https://www.recetasnestle.com.co/recetas/ensal...,1 cucharada de aceite (14 g) 1 berenje...,"1. Precalienta el horno a 180°C/360°F, 10...",None
449,"Ensalada de pollo a la mandarina, aguacate y m...",https://www.recetasnestle.com.co/recetas/ensal...,2 tazas de jugo de mandarina (500 ml) ...,1. Precalienta el horno a 180°C/360°F 10 ...,None
450,Ensalada de ahuyama confitada con almíbar de c...,https://www.recetasnestle.com.co/recetas/ensal...,1 cucharadita de jengibre fresco rallado (5...,"1. En una olla pon el agua, la citronela,...",None
451,"Ensalada de langostinos, piña y chips de pláta...",https://www.recetasnestle.com.co/recetas/ensal...,8 langostinos desvenados y limpios (300 g) ...,1. En un tazón marina los langostinos con...,None
452,Ensalada de terrina con frutas mixtas,https://www.recetasnestle.com.co/recetas/ensal...,1 mango tommy maduro (300 g) ½ libra d...,1. Pela el mango y con la ayuda de un cuc...,None
453,Ensalada de frutas navideña,https://www.recetasnestle.com.co/recetas/ensal...,1 cucharada al ras de azúcar (7 g) ½ l...,1. En una olla a fuego medio añade el azú...,None
454,"Ensalada de pollo, duraznos, rúgula, mozzarell...",https://www.recetasnestle.com.co/recetas/ensal...,1 pechuga de pollo sin piel y sin hueso par...,1. Saca las láminas de JUGOSO AL SARTÉN M...,None
455,"Ensalada de palmitos, rúgula y macarrones con ...",https://www.recetasnestle.com.co/recetas/ensal...,1 taza de macarrones cocinados al dente (25...,"1. En un tazón mezcla los macarrones, los...",None


## Store the dataset

In [18]:
df.to_csv('recipes.csv', index=False)